In [ ]:
%load_ext autoreload
%autoreload 2

# Loading Data

In [ ]:
from dataloading import get_all_data

In [ ]:
PATH_DATA = '../data/all_vectors.csv'
TEST_SIZE = .2
UNIQUE_COLUMNS = False

(X_train, X_test, y_train, y_test), (info_data, info_label) = get_all_data(
    path_all_vectors=PATH_DATA,
    test_size=TEST_SIZE,
    unique=UNIQUE_COLUMNS
    )

***

# Model

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# MODEL INIT
model = RandomForestClassifier(
    verbose=True,
    max_depth=5,
    max_features=0.1,
    n_estimators=1,
    bootstrap=True,
    oob_score=False,
    )

In [ ]:
# TRAINING

_ = model.fit(X_train, y_train)

In [ ]:
from collections import defaultdict

count = defaultdict(int)
for tree in model.estimators_:
    # print(tree.get_depth(), end=', ')
    count[tree.get_depth()] += 1
count = dict(count)
for item, key in count.items():
    print(f'{key} trees have length {item}')

In [ ]:
import numpy as np

# get predictions
y_pred = model.predict(X_test)

matches = np.count_nonzero(y_test == y_pred)
print(f'Accuracy: {100 * matches / len(y_test)} %')

In [ ]:
from prettytable import PrettyTable

table = PrettyTable()
table.field_names = ["Property", "Importance (%)"]

importances = model.feature_importances_
indices = np.argsort(importances)[::-1]

for i in indices:
    imp = 100 * importances[i]

    if imp > 0:
        if int(imp) == 0:
            imp = '<1 %'
        else:
            imp = f'{int(imp)} %'
    else:
        imp = '0 %'


    table.add_row([info_data[i], imp])

print(table)

# WanDB init

In [ ]:
# assert False, "WanDB not yet needed."

In [ ]:
import wandb

# start a new wandb run and add your model hyperparameters
wandb.init(project='Halado_Adatelemzes_Labor', config=model.get_params())

# Add additional configs to wandb
wandb.config.update({"test_size" : TEST_SIZE,
                    "train_len" : len(X_train),
                    "test_len" : len(X_test)})

In [ ]:
from wandb.sklearn import plot_precision_recall, plot_feature_importances
from wandb.sklearn import plot_class_proportions, plot_learning_curve, plot_roc

y_probas = model.predict_proba(X_test)

# log additional visualisations to wandb
plot_class_proportions(y_train, y_test, labels)
plot_learning_curve(model, X_train, y_train)
plot_roc(y_test, y_probas, labels)
plot_precision_recall(y_test, y_probas, labels)
plot_feature_importances(model)

In [ ]:
# Finish the wandb run
wandb.finish()